In [1]:
import dill
dill.load_session('case_value_session1.db')

In [1]:
import pandas as pd 
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error


In [2]:
path = 'C:/Users/isaac/Dropbox/Apps/ShareLaTeX/information_lawyer_quality'

# Read data 
data = pd.read_csv(path + '/_aux/hd_case_value.csv') 
#data = data.sample(frac=0.1, replace=True, random_state=1)
print(data.shape)
data.columns

(5356, 28)


Index(['gen', 'horas_sem', 'sal_caidos', 'prima_antig', 'prima_vac', 'rec20',
       'prima_dom', 'desc_sem', 'desc_ob', 'sarimssinf', 'utilidades',
       'nulidad', 'salario_diario', 'reinst', 'indem', 'horas_extra',
       'antiguedad', 'min_prima_antig', 'min_ag', 'min_vac', 'min_ley',
       'c_recsueldo', 'reclutamiento', 'num_actores', 'liq_total', 'duracion',
       'liq_total_disc', 'hd'],
      dtype='object')

In [3]:
data.describe()

,gen,horas_sem,sal_caidos,prima_antig,prima_vac,rec20,prima_dom,desc_sem,desc_ob,sarimssinf,...,min_ag,min_vac,min_ley,c_recsueldo,reclutamiento,num_actores,liq_total,duracion,liq_total_disc,hd
count,5340.000000,5256.000000,5340.000000,5340.000000,5334.000000,5341.000000,5341.000000,5341.000000,5341.000000,5341.000000,...,5286.000000,5209.000000,5.336000e+03,5.340000e+03,5198.000000,5356.000000,5.356000e+03,5356.000000,5.356000e+03,5356.000000
mean,0.483895,57.286975,0.982959,0.849438,0.955381,0.254634,0.200899,0.171691,0.290957,0.552893,...,2758.465764,2068.817676,5.341699e+04,2.172577e+04,0.195652,1.141337,3.080080e+04,28.801694,1.976041e+04,0.933906
std,0.499787,15.538088,0.129437,0.357655,0.206486,0.435696,0.400710,0.377147,0.454246,0.497241,...,14974.160132,6613.749970,1.078012e+05,2.429452e+05,0.396740,0.464623,1.437205e+05,153.506144,8.149673e+04,0.248470
min,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.470000,0.990000,0.000000e+00,0.000000e+00,0.000000,1.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
25%,0.000000,48.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,561.455365,361.970000,2.059550e+04,0.000000e+00,0.000000,1.000000,0.000000e+00,0.350000,0.000000e+00,1.000000
50%,0.000000,60.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,1282.189900,868.609990,3.248035e+04,1.077145e+03,0.000000,1.000000,9.000000e+03,0.760000,5.884490e+03,1.000000
75%,1.000000,66.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,...,2489.529900,1929.759300,5.545549e+04,7.475002e+03,0.000000,1.000000,2.216250e+04,1.630137,1.650361e+04,1.000000
max,1.000000,258.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,725185.750000,213154.980000,4.449881e+06,1.531981e+07,1.000000,3.000000,6.700000e+06,1396.000000,3.453441e+06,1.000000


In [4]:
X = data[['gen',
            'reclutamiento',
            'salario_diario',
            'horas_sem',
            'reinst',
            'prima_dom',
            'desc_sem',
            'sarimssinf',
            'antiguedad',
            'c_recsueldo',
            'desc_ob',
            'min_prima_antig',
            'min_ag',
            'min_vac',
            'min_ley']]

Z = data[['num_actores',
            'gen',
            'reclutamiento',
            'salario_diario',
            'antiguedad',          
            'horas_sem',
            'reinst',
            'indem',
            'sal_caidos',
            'prima_antig',
            'prima_vac',
            'horas_extra',
            'rec20',
            'prima_dom',
            'desc_sem',
            'sarimssinf',
            'utilidades',
            'nulidad']]

y1 = data['liq_total']
y2 = data['liq_total_disc']

In [5]:
numeric_featuresX = ['salario_diario',
            'horas_sem',
            'antiguedad',
            'c_recsueldo',
            'min_prima_antig',
            'min_ag',
            'min_vac',
            'min_ley']
numeric_featuresZ = ['salario_diario',
            'horas_sem',
            'antiguedad']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('polynomial', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())])


categorical_featuresZ = ['num_actores']
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse = False)


binary_featuresX = ['gen',
            'reclutamiento',
            'reinst',
            'prima_dom',
            'desc_sem',
            'sarimssinf',
            'desc_ob']
binary_featuresZ = ['gen',
            'reclutamiento',
            'reinst',
            'indem',
            'sal_caidos',
            'prima_antig',
            'prima_vac',
            'horas_extra',
            'rec20',
            'prima_dom',
            'desc_sem',
            'sarimssinf',
            'utilidades',
            'nulidad']
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('polynomial', PolynomialFeatures(degree=1, include_bias=False))])


preprocessorX = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_featuresX),
        ('bin', binary_transformer, binary_featuresX)])
preprocessorZ = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_featuresZ),
        ('cat', categorical_transformer, categorical_featuresZ),
        ('bin', binary_transformer, binary_featuresZ)])


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Z_train, Z_test, y_train, y_test = train_test_split(X, Z, y2, test_size=0.2,
                                                    random_state=0)

In [7]:
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
from sklearn.linear_model import LinearRegression

In [8]:
# Linear Regression
LRX = Pipeline(steps=[('preprocessor', preprocessorX),
                      ('model', LinearRegression())])
LRX.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['salario_diario',
                                                   'horas_sem', 'antiguedad',
                                                   'c_recsueldo',
                                                   'min_prima_antig', 'min_ag',
                                                   'min_vac', 'min_ley']),
        

In [9]:
LRZ = Pipeline(steps=[('preprocessor', preprocessorZ),
                      ('model', LinearRegression())])
LRZ.fit(Z_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['salario_diario',
                                                   'horas_sem', 'antiguedad']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
  

In [10]:
from sklearn.ensemble import GradientBoostingRegressor

In [11]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'model__n_estimators'  : [50, 100, 500, 1000],
              'model__max_features'  : ['auto', 'sqrt', 'log2'],
              'model__max_depth'     : [None, 1, 3, 5, 10, 20],
              'model__subsample'     : [0.5, 1],
              'model__learning_rate' : [0.001, 0.01, 0.1]
             }


GB  = GradientBoostingRegressor(
                        random_state        = 123,
                        # Activación de la parada temprana
                        validation_fraction = 0.1,
                        n_iter_no_change    = 5,
                        tol                 = 0.0001
                    )
# Gradient Boosting
GBX = Pipeline(steps=[('preprocessor', preprocessorX),
                      ('model', GB)])
GBZ = Pipeline(steps=[('preprocessor', preprocessorZ),
                      ('model', GB)])

# Búsqueda por grid search con validación cruzada
# ==============================================================================
gridX = GridSearchCV(GBX,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 4,
        return_train_score = True
       )
gridZ = GridSearchCV(GBZ,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 4,
        return_train_score = True
       )

gridX.fit(X = X_train, y = y_train)
gridZ.fit(X = Z_train, y = y_train)

Fitting 15 folds for each of 432 candidates, totalling 6480 fits
Fitting 15 folds for each of 432 candidates, totalling 6480 fits


GridSearchCV(cv=RepeatedKFold(n_repeats=3, n_splits=5, random_state=123),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('polynomial',
                                                                                          PolynomialFeatures(include_bias=False)),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['salario_diario',
                          

In [12]:
# Resultados
# ==============================================================================
resultados = pd.DataFrame(gridX.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

,param_model__learning_rate,param_model__max_depth,param_model__max_features,param_model__n_estimators,param_model__subsample,mean_test_score,std_test_score,mean_train_score,std_train_score
158,0.01,None,sqrt,1000,0.5,-63069.171309,20991.393747,-54502.803082,7438.463315
156,0.01,None,sqrt,500,0.5,-63069.171309,20991.393747,-54502.803082,7438.463315
154,0.01,None,sqrt,100,0.5,-63069.171309,20991.393747,-54502.803082,7438.463315
272,0.01,20,sqrt,50,0.5,-63125.498467,20571.390753,-56638.292983,7707.608457


In [13]:
modelo_final_GBX = gridX.best_estimator_
modelo_final_GBZ = gridZ.best_estimator_

In [14]:
importancia = permutation_importance(
                estimator    = modelo_final_GBX,
                X            = X_train,
                y            = y_train,
                n_repeats    = 5,
                scoring      = 'neg_root_mean_squared_error',
                n_jobs       = multiprocessing.cpu_count() - 1,
                random_state = 123
             )

# Se almacenan los resultados (media y desviación) en un dataframe
df_importancia = pd.DataFrame(
                    {k: importancia[k] for k in ['importances_mean', 'importances_std']}
                 )
df_importancia['feature'] = X_train.columns
df_importancia.sort_values('importances_mean', ascending=False)

,importances_mean,importances_std,feature
2,1369.923209,30.428258,salario_diario
8,1272.040208,76.346678,antiguedad
13,1063.813219,41.270977,min_vac
14,1021.884767,26.354959,min_ley
9,932.458964,37.691371,c_recsueldo
12,625.800035,46.292649,min_ag
11,395.191065,79.743210,min_prima_antig
3,349.643261,84.362845,horas_sem
0,12.981643,5.885754,gen
7,10.458658,7.642260,sarimssinf


In [15]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

In [16]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'model__loss'             : ['least_squares', 'least_absolute_deviation'],
              'model__learning_rate'    : [0.001, 0.01, 0.1],
              'model__max_depth'        : [None, 1, 3, 5, 10, 20],
              'model__l2_regularization': [0, 1]
             }


# Gradient Boosting
HGBX = Pipeline(steps=[('preprocessor', preprocessorX),
                      ('model', HistGradientBoostingRegressor(
                        max_iter            = 10000,
                        # Activación de la parada temprana
                        early_stopping      = True,
                        scoring             = 'loss',
                        validation_fraction = 0.1,
                        n_iter_no_change    = 10,
                        tol                 = 1e-7,
                        random_state        = 123
                    ))])
HGBZ = Pipeline(steps=[('preprocessor', preprocessorZ),
                      ('model', HistGradientBoostingRegressor(
                        max_iter            = 10000,
                        # Activación de la parada temprana
                        early_stopping      = True,
                        scoring             = 'loss',
                        validation_fraction = 0.1,
                        n_iter_no_change    = 10,
                        tol                 = 1e-7,
                        random_state        = 123
                    ))])

# Búsqueda por grid search con validación cruzada
# ==============================================================================
gridX = GridSearchCV(HGBX,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 4,
        return_train_score = True
       )
gridZ = GridSearchCV(HGBZ,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 4,
        return_train_score = True
       )

gridX.fit(X = X_train, y = y_train)
gridZ.fit(X = Z_train, y = y_train)

# Resultados
# ==============================================================================
resultados = pd.DataFrame(gridX.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

Fitting 15 folds for each of 72 candidates, totalling 1080 fits
Fitting 15 folds for each of 72 candidates, totalling 1080 fits


,param_model__l2_regularization,param_model__learning_rate,param_model__loss,param_model__max_depth,mean_test_score,std_test_score,mean_train_score,std_train_score
17,0,0.01,least_squares,20,-62004.285236,20479.769375,-59164.501226,5987.460466
12,0,0.01,least_squares,None,-62007.484829,20487.917194,-59153.487344,5995.054509
53,1,0.01,least_squares,20,-62069.271694,20485.313862,-58762.459814,5997.236032
48,1,0.01,least_squares,None,-62080.375746,20477.548613,-58796.357092,5921.683080


In [17]:
modelo_final_HGBX = gridX.best_estimator_
modelo_final_HGBZ = gridZ.best_estimator_

In [18]:
from xgboost import XGBRegressor


In [19]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'model__max_depth'        : [None, 1, 3, 5, 10, 20],
              'model__subsample'        : [0.5, 1],
              'model__learning_rate'    : [0.001, 0.01, 0.1],
              'model__booster'          : ['gbtree'],
              'model__n_estimators'     : [50, 100, 500, 1000],
             }


# Crear conjunto de validación
# ==============================================================================
np.random.seed(123)
idx_validacion = np.random.choice(
                    X_train.shape[0],
                    size= int(X_train.shape[0]*0.1),
                    replace=False
                 )

In [20]:
X_val = X_train.iloc[idx_validacion, :].copy()
Z_val = Z_train.iloc[idx_validacion, :].copy()
y_val = y_train.iloc[idx_validacion].copy()

In [21]:
X_train_grid = X_train.reset_index(drop = True).drop(idx_validacion, axis = 0).copy()
Z_train_grid = Z_train.reset_index(drop = True).drop(idx_validacion, axis = 0).copy()
y_train_grid = y_train.reset_index(drop = True).drop(idx_validacion, axis = 0).copy()

In [22]:
preprocessorX_ = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_featuresX),
        ('bin', binary_transformer, binary_featuresX)],
        remainder='passthrough')
preprocessorZ_ = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_featuresZ),
        ('cat', categorical_transformer, categorical_featuresZ),
        ('bin', binary_transformer, binary_featuresZ)],
        remainder='passthrough')

In [23]:
encoderX = preprocessorX_.fit(X_train)
encoderZ = preprocessorZ_.fit(Z_train)

In [24]:
X_train_grid = encoderX.transform(X_train_grid)
Z_train_grid = encoderZ.transform(Z_train_grid)

In [25]:
X_val = encoderX.transform(X_val)
Z_val = encoderZ.transform(Z_val)

In [26]:
# XGBoost necesita pasar los paramétros específicos del entrenamiento al llamar
# al método .fit()
fit_paramsX = {"model__early_stopping_rounds" : 5, 
              "model__eval_metric"           : "rmse", 
              "model__eval_set"              : [(X_val, y_val)],
              "model__verbose"               : 0
             }
fit_paramsZ = {"model__early_stopping_rounds" : 5, 
              "model__eval_metric"           : "rmse", 
              "model__eval_set"              : [(Z_val, y_val)],
              "model__verbose"               : 0
             }

In [27]:
XGBX = Pipeline(steps=[('model', XGBRegressor(
                        random_state = 123
                    ))])
XGBZ = Pipeline(steps=[('model', XGBRegressor(
                        random_state = 123
                    ))])

In [28]:
# Búsqueda por grid search con validación cruzada
# ==============================================================================
gridX = GridSearchCV(
        estimator  = XGBX,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 4,
        return_train_score = True
       )
gridZ = GridSearchCV(
        estimator  = XGBZ,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 4,
        return_train_score = True
       )

In [29]:
gridX.fit(X = X_train_grid, y = y_train_grid, **fit_paramsX)
gridZ.fit(X = Z_train_grid, y = y_train_grid, **fit_paramsZ)

Fitting 15 folds for each of 144 candidates, totalling 2160 fits
Fitting 15 folds for each of 144 candidates, totalling 2160 fits


GridSearchCV(cv=RepeatedKFold(n_repeats=3, n_splits=5, random_state=123),
             estimator=Pipeline(steps=[('model',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                     gamma=None, gpu_id=None,
                                                     importance_type='gain',
                                                     interaction_constraints=None,
                                                     learning_rate=None,
                                                     max_delta_step=None,
                                                     max_depth=None,
                                                     min_

In [30]:
# Resultados
# ==============================================================================
resultados = pd.DataFrame(gridX.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

,param_model__booster,param_model__learning_rate,param_model__max_depth,param_model__n_estimators,param_model__subsample,mean_test_score,std_test_score,mean_train_score,std_train_score
119,gbtree,0.1,3,1000,1,-66433.003375,22774.945189,-62386.853167,7416.332267
117,gbtree,0.1,3,500,1,-66433.003375,22774.945189,-62386.853167,7416.332267
115,gbtree,0.1,3,100,1,-66433.003375,22774.945189,-62386.853167,7416.332267
113,gbtree,0.1,3,50,1,-66433.003375,22774.945189,-62386.853167,7416.332267


In [31]:
modelo_final_XGBX = gridX.best_estimator_
modelo_final_XGBZ = gridZ.best_estimator_

In [32]:
from sklearn.ensemble import RandomForestRegressor


In [33]:
param_grid = {'model__n_estimators': [100, 200, 500],
              'model__max_features': ["auto", "sqrt", "log2"],
              'model__max_depth'   : [None, 3, 10, 20],
              'model__criterion'   : ["mse", "mae"]
             }

# Random Forests
RFX = Pipeline(steps=[('preprocessor', preprocessorX),
                      ('model', RandomForestRegressor(random_state=123))])
RFZ = Pipeline(steps=[('preprocessor', preprocessorZ),
                      ('model', RandomForestRegressor(random_state=123))])

# Búsqueda por grid search con validación cruzada
# ==============================================================================
gridX = GridSearchCV(
        estimator  = RFX,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 4,
        return_train_score = True
       )
gridZ = GridSearchCV(
        estimator  = RFZ,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 4,
        return_train_score = True
       )

gridX.fit(X = X_train, y = y_train)
gridZ.fit(X = Z_train, y = y_train)

# Resultados
# ==============================================================================
resultados = pd.DataFrame(gridX.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

Fitting 15 folds for each of 72 candidates, totalling 1080 fits
Fitting 15 folds for each of 72 candidates, totalling 1080 fits


,param_model__criterion,param_model__max_depth,param_model__max_features,param_model__n_estimators,mean_test_score,std_test_score,mean_train_score,std_train_score
15,mse,3,log2,100,-65576.848773,21402.003828,-48389.587037,3346.247156
17,mse,3,log2,500,-65585.733666,21587.838127,-48261.791965,3159.783166
42,mae,None,log2,100,-65622.080940,21584.480350,-26702.862047,3533.386616
24,mse,10,log2,100,-65633.975781,21115.154903,-30663.165575,3249.604810


In [34]:
modelo_final_RFX = gridX.best_estimator_
modelo_final_RFZ = gridZ.best_estimator_

In [35]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.cross_decomposition import PLSRegression
#from sklearn.pipeline import make_pipeline

In [36]:
# PCR
pcrX = Pipeline(steps=[('preprocessor', preprocessorX),
                     ('components', TruncatedSVD(n_components=10)),
                      ('model', LinearRegression())])
pcrZ = Pipeline(steps=[('preprocessor', preprocessorZ),
                     ('components', TruncatedSVD(n_components=10)),
                      ('model', LinearRegression())])

pcrX.fit(X_train, y_train)
pcrZ.fit(Z_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['salario_diario',
                                                   'horas_sem', 'antiguedad']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
  

In [37]:
from sklearn.preprocessing import FunctionTransformer

plsX = Pipeline(steps=[('preprocessor', preprocessorX),
                    ('model', PLSRegression(n_components=10))])
plsZ = Pipeline(steps=[('preprocessor', preprocessorZ),
                    ('model', PLSRegression(n_components=10))])



In [38]:
plsX.fit(X_train, y_train)
plsZ.fit(Z_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['salario_diario',
                                                   'horas_sem', 'antiguedad']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
  

In [39]:
from sklearn.neural_network import MLPRegressor

In [40]:
# Espacio de búsqueda de cada hiperparámetro
# ==============================================================================
param_grid = {
    'modelo__hidden_layer_sizes': [(10), (20), (10, 10), (100,), (50,50,50), (10,20,10,20)],
    'modelo__alpha': [0.0001, 0.001, 0.01],
    'modelo__learning_rate_init': [0.001, 0.01, 0.1, 1],
    'modelo__learning_rate': ['constant', 'adaptive'],
    'modelo__solver': ['lbfgs', 'adam'],
}

# Se combinan los pasos de preprocesado y el modelo en un mismo pipeline
MLPX = Pipeline([('preprocessing', preprocessorX),
                 ('modelo', MLPRegressor(max_iter=10000, random_state=123, verbose = True))])
MLPZ = Pipeline([('preprocessing', preprocessorZ),
                 ('modelo', MLPRegressor(max_iter=10000, random_state=123, verbose = True))])

# Búsqueda por validación cruzada
# ==============================================================================
gridX = GridSearchCV(
        estimator  = MLPX,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 1,
        return_train_score = True
       )
gridZ = GridSearchCV(
        estimator  = MLPZ,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 1,
        return_train_score = True
       )

gridX.fit(X = X_train, y = y_train)
gridZ.fit(X = Z_train, y = y_train)

# Resultados
# ==============================================================================
resultados = pd.DataFrame(gridX.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

Fitting 15 folds for each of 288 candidates, totalling 4320 fits
Iteration 1, loss = 2724755322.98816109
Iteration 2, loss = 2510905567.62352467
Iteration 3, loss = 2510563945.29874945
Iteration 4, loss = 2510209491.26505947
Iteration 5, loss = 2509836324.05729246
Iteration 6, loss = 2509470050.22171783
Iteration 7, loss = 2509098882.58382559
Iteration 8, loss = 2508720854.41511488
Iteration 9, loss = 2508335939.64172077
Iteration 10, loss = 2507957552.95315552
Iteration 11, loss = 2507576316.79245615
Iteration 12, loss = 2507192308.70005798
Iteration 13, loss = 2506803818.04110289
Iteration 14, loss = 2506418807.21482182
Iteration 15, loss = 2506039683.08998108
Iteration 16, loss = 2505647681.40948439
Iteration 17, loss = 2505269124.08581877
Iteration 18, loss = 2504881401.21202183
Iteration 19, loss = 2504491052.84546518
Iteration 20, loss = 2504102144.05731964
Iteration 21, loss = 2503714536.51567364
Iteration 22, loss = 2503337148.56002045
Iteration 23, loss = 2502951748.42273378
I

Iteration 198, loss = 2443744717.82070112
Iteration 199, loss = 2443452814.50682211
Iteration 200, loss = 2443151566.83743668
Iteration 201, loss = 2442856848.03404951
Iteration 202, loss = 2442571868.65741062
Iteration 203, loss = 2442278011.55540752
Iteration 204, loss = 2441981567.35803986
Iteration 205, loss = 2441692082.30171347
Iteration 206, loss = 2441393321.97740126
Iteration 207, loss = 2441100619.44400263
Iteration 208, loss = 2440813788.57672071
Iteration 209, loss = 2440517059.67179441
Iteration 210, loss = 2440228015.70899916
Iteration 211, loss = 2439940463.26252031
Iteration 212, loss = 2439654069.17013884
Iteration 213, loss = 2439362430.91926241
Iteration 214, loss = 2439084506.19746828
Iteration 215, loss = 2438785490.61516523
Iteration 216, loss = 2438503624.95822287
Iteration 217, loss = 2438222062.82525539
Iteration 218, loss = 2437925335.72946262
Iteration 219, loss = 2437642641.13751745
Iteration 220, loss = 2437357981.68183947
Iteration 221, loss = 2437068534.0

Iteration 394, loss = 2394425753.28354549
Iteration 395, loss = 2394212897.04740667
Iteration 396, loss = 2394007877.39681292
Iteration 397, loss = 2393802660.59064245
Iteration 398, loss = 2393590654.31710339
Iteration 399, loss = 2393392117.18704462
Iteration 400, loss = 2393190320.51495790
Iteration 401, loss = 2392981447.99016666
Iteration 402, loss = 2392772643.52799129
Iteration 403, loss = 2392572468.17439747
Iteration 404, loss = 2392364669.16433334
Iteration 405, loss = 2392158157.84931660
Iteration 406, loss = 2391959056.51485872
Iteration 407, loss = 2391750573.29265499
Iteration 408, loss = 2391551351.01231766
Iteration 409, loss = 2391342863.87258101
Iteration 410, loss = 2391152849.65472126
Iteration 411, loss = 2390951332.21271896
Iteration 412, loss = 2390750285.37234306
Iteration 413, loss = 2390548043.49475193
Iteration 414, loss = 2390357902.02486515
Iteration 415, loss = 2390161852.20720530
Iteration 416, loss = 2389961745.44466114
Iteration 417, loss = 2389757289.0

Iteration 590, loss = 2361839993.22214842
Iteration 591, loss = 2361715553.45690680
Iteration 592, loss = 2361589638.31423903
Iteration 593, loss = 2361466111.70687199
Iteration 594, loss = 2361328332.28052187
Iteration 595, loss = 2361199913.13658619
Iteration 596, loss = 2361076708.33229446
Iteration 597, loss = 2360945744.15885687
Iteration 598, loss = 2360825479.10507298
Iteration 599, loss = 2360698339.05166674
Iteration 600, loss = 2360577536.29984188
Iteration 601, loss = 2360449949.20761204
Iteration 602, loss = 2360321752.83780956
Iteration 603, loss = 2360206370.98674202
Iteration 604, loss = 2360082668.60497618
Iteration 605, loss = 2359961393.77771235
Iteration 606, loss = 2359840241.55054045
Iteration 607, loss = 2359710392.95277166
Iteration 608, loss = 2359589516.76774359
Iteration 609, loss = 2359463856.01667690
Iteration 610, loss = 2359341544.52699947
Iteration 611, loss = 2359226013.52069759
Iteration 612, loss = 2359101687.72726059
Iteration 613, loss = 2358990825.4

Iteration 787, loss = 2343285867.58434153
Iteration 788, loss = 2343224733.58830261
Iteration 789, loss = 2343165276.02403307
Iteration 790, loss = 2752761190.78140879
Iteration 791, loss = 2343039349.69968081
Iteration 792, loss = 2342980666.46601152
Iteration 793, loss = 2342917883.92935514
Iteration 794, loss = 2342859241.99403143
Iteration 795, loss = 2342802945.15462208
Iteration 796, loss = 2342739832.45064735
Iteration 797, loss = 2342683980.94919968
Iteration 798, loss = 2342618480.73658466
Iteration 799, loss = 2342563743.73680067
Iteration 800, loss = 2342503842.84371519
Iteration 801, loss = 2342445112.39001560
Iteration 802, loss = 2342383379.87918711
Iteration 803, loss = 2342326343.76013517
Iteration 804, loss = 2342266829.75582027
Iteration 805, loss = 2342208465.07583904
Iteration 806, loss = 2342152371.56296587
Iteration 807, loss = 2342095055.13643551
Iteration 808, loss = 2342037056.01262617
Iteration 809, loss = 2341986327.46091080
Iteration 810, loss = 2341927544.3

Iteration 983, loss = 2337547548.58810520
Iteration 984, loss = 2337522554.08314323
Iteration 985, loss = 2337499166.67186213
Iteration 986, loss = 2337472899.62121916
Iteration 987, loss = 2337447575.76877165
Iteration 988, loss = 2337422196.41962576
Iteration 989, loss = 2337396860.99498463
Iteration 990, loss = 2337373769.04878950
Iteration 991, loss = 2337349623.68733788
Iteration 992, loss = 2337323159.15534019
Iteration 993, loss = 2337297495.43054962
Iteration 994, loss = 2358459782.12556744
Iteration 995, loss = 2337246843.79113102
Iteration 996, loss = 2337222982.41159773
Iteration 997, loss = 2337195636.58689070
Iteration 998, loss = 2337168981.83505106
Iteration 999, loss = 2337145206.06468630
Iteration 1000, loss = 2337118327.82799435
Iteration 1001, loss = 2337095010.60960913
Iteration 1002, loss = 2337071968.57528305
Iteration 1003, loss = 2337046634.82960892
Iteration 1004, loss = 2337024096.98660755
Iteration 1005, loss = 2336998780.94068670
Iteration 1006, loss = 23369

,param_modelo__alpha,param_modelo__hidden_layer_sizes,param_modelo__learning_rate,param_modelo__learning_rate_init,param_modelo__solver,mean_test_score,std_test_score,mean_train_score,std_train_score
271,0.01,"(50, 50, 50)",adaptive,1,adam,-65106.581212,21125.295547,-67996.679809,5672.079262
263,0.01,"(50, 50, 50)",constant,1,adam,-65106.581212,21125.295547,-67996.679809,5672.079262
287,0.01,"(10, 20, 10, 20)",adaptive,1,adam,-65238.750999,21493.755040,-68417.917729,5420.264358
279,0.01,"(10, 20, 10, 20)",constant,1,adam,-65238.750999,21493.755040,-68417.917729,5420.264358


In [41]:
modelo_final_MLPX = gridX.best_estimator_
modelo_final_MLPZ = gridZ.best_estimator_

In [42]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from scipy.stats import uniform

In [43]:
# Espacio de búsqueda de cada hiperparámetro
# ==============================================================================
param_distributions = {
    'modelo__alpha': uniform(0.001, 10) ,
    'modelo__l1_ratio' : uniform(0.019, 0.99)
}

# Se combinan los pasos de preprocesado y el modelo en un mismo pipeline
ENX = Pipeline([('preprocessing', preprocessorX),
                 ('modelo', ElasticNet(random_state=123))])

ENZ = Pipeline([('preprocessing', preprocessorZ),
                 ('modelo', ElasticNet(random_state=123))])


# Búsqueda por validación cruzada
# ==============================================================================
gridX = RandomizedSearchCV(
        estimator  = ENX,
        param_distributions = param_distributions,
        n_iter     = 1000,
        scoring    = 'neg_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123),  
        verbose    = 4,
        random_state = 123,
        return_train_score = True
       )
gridZ = RandomizedSearchCV(
        estimator  = ENZ,
        param_distributions = param_distributions,
        n_iter     = 1000,
        scoring    = 'neg_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123),  
        verbose    = 4,
        random_state = 123,
        return_train_score = True
       )

gridX.fit(X = X_train, y = y_train)
gridZ.fit(X = Z_train, y = y_train)

# Resultados
# ==============================================================================
resultados = pd.DataFrame(gridX.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

Fitting 15 folds for each of 1000 candidates, totalling 15000 fits


C:\Users\isaac\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [-6.65450484e+09 -1.11860649e+10 -6.98902063e+09 -7.61088810e+09
 -7.70202692e+09 -1.17463523e+10 -6.98970499e+09 -1.11366760e+10
 -9.50293709e+09 -8.08372245e+09 -1.17490989e+10 -7.80302753e+09
 -6.68453993e+09 -7.79747546e+09 -1.08967195e+10 -1.61527848e+10
 -8.41040312e+09 -7.69431936e+09 -1.86277038e+10 -6.76304245e+09
 -6.47100533e+09 -8.84946607e+09 -6.20937654e+09 -1.28160807e+11
 -8.64457482e+09 -3.28601204e+10 -7.85609490e+09 -8.17144462e+09
 -1.00264660e+10 -6.93413332e+09 -7.83757084e+09 -8.62689257e+09
 -5.94505776e+09 -6.39776410e+09 -1.23526871e+10 -5.55785800e+10
 -9.10756273e+09 -7.65505930e+10 -6.75753758e+09 -1.75835789e+10
 -1.14229094e+10 -7.38787838e+09 -9.76805843e+09 -7.35839050e+09
 -8.88409101e+09             nan -1.23504955e+10 -8.93579095e+09
 -1.17725427e+10 -9.34441445e+09 -9.13394195e+09 -1.16705292e+10
 -1.08817951

Fitting 15 folds for each of 1000 candidates, totalling 15000 fits


C:\Users\isaac\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [-5.13328242e+09 -7.66569018e+09 -5.29907931e+09 -5.65279164e+09
 -5.70807651e+09 -7.90527107e+09 -5.29934000e+09 -7.64450637e+09
 -6.81323125e+09 -5.94557041e+09 -7.90813851e+09 -5.77027423e+09
 -5.14731680e+09 -5.76672232e+09 -7.53453958e+09 -9.28331330e+09
 -6.15170444e+09 -5.70334974e+09 -9.83083306e+09 -5.18482312e+09
 -5.05140307e+09 -6.42524069e+09 -4.94663546e+09 -1.53012678e+10
 -6.29876468e+09 -1.17454401e+10 -5.80312461e+09 -6.00079343e+09
 -7.10241543e+09 -5.27044321e+09 -5.79165127e+09 -6.28791299e+09
 -4.85550421e+09 -5.02064296e+09 -8.14187331e+09 -1.33039547e+10
 -6.58245223e+09 -1.41488070e+10 -5.18212569e+09 -9.61159724e+09
 -7.76959740e+09 -5.52038829e+09 -6.96380057e+09 -5.50317946e+09
 -6.44645606e+09             nan -8.14212251e+09 -6.47854936e+09
 -7.91480895e+09 -6.72184996e+09 -6.59805968e+09 -7.87268246e+09
 -7.52772831

,param_modelo__alpha,param_modelo__l1_ratio,mean_test_score,std_test_score,mean_train_score,std_train_score
963,9.818287,0.022513,-5.625082e+09,4.398817e+09,-3.965096e+09,7.262548e+08
237,9.990184,0.05921,-5.651473e+09,4.442843e+09,-3.959022e+09,7.260700e+08
947,9.663854,0.038339,-5.665874e+09,4.466946e+09,-3.955735e+09,7.259637e+08
761,9.557926,0.063832,-5.714966e+09,4.549601e+09,-3.944703e+09,7.255771e+08


In [44]:
modelo_final_ENX = gridX.best_estimator_
modelo_final_ENZ = gridZ.best_estimator_

In [45]:
# Espacio de búsqueda de cada hiperparámetro
# ==============================================================================
param_distributions = {
    'modelo__alpha': uniform(0.001, 10)
}

# Se combinan los pasos de preprocesado y el modelo en un mismo pipeline
LASSOX = Pipeline([('preprocessing', preprocessorX),
                 ('modelo', Lasso(random_state=123))])

LASSOZ = Pipeline([('preprocessing', preprocessorZ),
                 ('modelo', ElasticNet(random_state=123))])


# Búsqueda por validación cruzada
# ==============================================================================
gridX = RandomizedSearchCV(
        estimator  = LASSOX,
        param_distributions = param_distributions,
        n_iter     = 1000,
        scoring    = 'neg_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123),  
        verbose    = 4,
        random_state = 123,
        return_train_score = True
       )
gridZ = RandomizedSearchCV(
        estimator  = LASSOZ,
        param_distributions = param_distributions,
        n_iter     = 1000,
        scoring    = 'neg_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123),  
        verbose    = 4,
        random_state = 123,
        return_train_score = True
       )

gridX.fit(X = X_train, y = y_train)
gridZ.fit(X = Z_train, y = y_train)

# Resultados
# ==============================================================================
resultados = pd.DataFrame(gridX.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

Fitting 15 folds for each of 1000 candidates, totalling 15000 fits


C:\Users\isaac\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5405324228832.488, tolerance: 1998996785.7667134
  model = cd_fast.enet_coordinate_descent(


Fitting 15 folds for each of 1000 candidates, totalling 15000 fits


,param_modelo__alpha,mean_test_score,std_test_score,mean_train_score,std_train_score
494,0.001819,-2.070410e+11,3.663949e+11,-2.395520e+09,2.539667e+08
671,0.021919,-2.070425e+11,3.663816e+11,-2.395522e+09,2.539672e+08
118,0.027881,-2.070429e+11,3.663777e+11,-2.395522e+09,2.539673e+08
320,0.036322,-2.070435e+11,3.663721e+11,-2.395523e+09,2.539675e+08


In [46]:
modelo_final_LASSOX = gridX.best_estimator_
modelo_final_LASSOZ = gridZ.best_estimator_

In [47]:
# Se combinan los pasos de preprocesado y el modelo en un mismo pipeline
RIDGEX = Pipeline([('preprocessing', preprocessorX),
                 ('modelo', Ridge())])

RIDGEZ = Pipeline([('preprocessing', preprocessorZ),
                 ('modelo', Ridge())])


# Búsqueda por validación cruzada
# ==============================================================================
gridX = RandomizedSearchCV(
        estimator  = RIDGEX,
        param_distributions = param_distributions,
        n_iter     = 1000,
        scoring    = 'neg_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123),  
        verbose    = 4,
        random_state = 123,
        return_train_score = True
       )
# ==============================================================================
gridZ = RandomizedSearchCV(
        estimator  = RIDGEZ,
        param_distributions = param_distributions,
        n_iter     = 1000,
        scoring    = 'neg_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123),  
        verbose    = 4,
        random_state = 123,
        return_train_score = True
       )

gridX.fit(X = X_train, y = y_train)
gridZ.fit(X = Z_train, y = y_train)

# Resultados
# ==============================================================================
resultados = pd.DataFrame(gridX.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

Fitting 15 folds for each of 1000 candidates, totalling 15000 fits
Fitting 15 folds for each of 1000 candidates, totalling 15000 fits


,param_modelo__alpha,mean_test_score,std_test_score,mean_train_score,std_train_score
474,9.990184,-2.291501e+11,3.767050e+11,-2.435820e+09,2.684508e+08
203,9.961863,-2.292111e+11,3.767843e+11,-2.435721e+09,2.684155e+08
637,9.959611,-2.292159e+11,3.767906e+11,-2.435713e+09,2.684127e+08
696,9.958818,-2.292176e+11,3.767928e+11,-2.435711e+09,2.684117e+08


In [5]:
# Resultados
# ==============================================================================
resultados = pd.DataFrame(gridX.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

,param_modelo__alpha,mean_test_score,std_test_score,mean_train_score,std_train_score
474,9.990184,-2.291501e+11,3.767050e+11,-2.435820e+09,2.684508e+08
203,9.961863,-2.292111e+11,3.767843e+11,-2.435721e+09,2.684155e+08
637,9.959611,-2.292159e+11,3.767906e+11,-2.435713e+09,2.684127e+08
696,9.958818,-2.292176e+11,3.767928e+11,-2.435711e+09,2.684117e+08


In [48]:
modelo_final_RIDGEX = gridX.best_estimator_
modelo_final_RIDGEZ = gridZ.best_estimator_

In [ ]:
import dill

In [9]:
hola =3

In [33]:
dill.dump_session('case_value_session2.db')

In [4]:
explained_variance_score(y_test, modelo_final_XGBX.predict(X_test_grid), multioutput='uniform_average')

NameError: name 'X_test_grid' is not defined

In [3]:
#Metric Results

metric_results_oosX = pd.DataFrame({'Linear Regression' :  [explained_variance_score(y_test, LRX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, LRX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, LRX.predict(X_test)),
    mean_squared_error(y_test, LRX.predict(X_test)),
    mean_absolute_percentage_error(y_test, LRX.predict(X_test))], 
                                     'GB' :  [explained_variance_score(y_test, modelo_final_GBX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_GBX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_GBX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_GBX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_GBX.predict(X_test))],
                                     'HGB' :  [explained_variance_score(y_test, modelo_final_HGBX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_HGBX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_HGBX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_HGBX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_HGBX.predict(X_test))],
                                    'RF' :  [explained_variance_score(y_test, modelo_final_RFX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_RFX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_RFX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_RFX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_RFX.predict(X_test))],
                                    'PC Regression' :  [explained_variance_score(y_test, pcrX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, pcrX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, pcrX.predict(X_test)),
    mean_squared_error(y_test, pcrX.predict(X_test)),
    mean_absolute_percentage_error(y_test, pcrX.predict(X_test))], 
                                    'PLS' :  [explained_variance_score(y_test, plsX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, plsX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, plsX.predict(X_test)),
    mean_squared_error(y_test, plsX.predict(X_test)),
    mean_absolute_percentage_error(y_test, plsX.predict(X_test))], 
                                    'MLP' :  [explained_variance_score(y_test, modelo_final_MLPX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_MLPX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_MLPX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_MLPX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_MLPX.predict(X_test))],
                                     'ElasticNet' :  [explained_variance_score(y_test, modelo_final_ENX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_ENX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_ENX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_ENX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_ENX.predict(X_test))],
                                     'LASSO' :  [explained_variance_score(y_test, modelo_final_LASSOX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_LASSOX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_LASSOX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_LASSOX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_LASSOX.predict(X_test))],
                                     'RIDGE' :  [explained_variance_score(y_test, modelo_final_RIDGEX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_RIDGEX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_RIDGEX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_RIDGEX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_RIDGEX.predict(X_test))]                                 
                        })    

In [4]:
metric_results_insX = pd.DataFrame({'Linear Regression' :  [explained_variance_score(y_train, LRX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, LRX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, LRX.predict(X_train)),
    mean_squared_error(y_train, LRX.predict(X_train)),
    mean_absolute_percentage_error(y_train, LRX.predict(X_train))], 
                                     'GB' :  [explained_variance_score(y_train, modelo_final_GBX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_GBX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_GBX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_GBX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_GBX.predict(X_train))],
                                     'HGB' :  [explained_variance_score(y_train, modelo_final_HGBX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_HGBX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_HGBX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_HGBX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_HGBX.predict(X_train))],
                                    'RF' :  [explained_variance_score(y_train, modelo_final_RFX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_RFX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_RFX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_RFX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_RFX.predict(X_train))],
                                    'PC Regression' :  [explained_variance_score(y_train, pcrX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, pcrX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, pcrX.predict(X_train)),
    mean_squared_error(y_train, pcrX.predict(X_train)),
    mean_absolute_percentage_error(y_train, pcrX.predict(X_train))], 
                                    'PLS' :  [explained_variance_score(y_train, plsX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, plsX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, plsX.predict(X_train)),
    mean_squared_error(y_train, plsX.predict(X_train)),
    mean_absolute_percentage_error(y_train, plsX.predict(X_train))], 
                                    'MLP' :  [explained_variance_score(y_train, modelo_final_MLPX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_MLPX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_MLPX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_MLPX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_MLPX.predict(X_train))],
                                     'ElasticNet' :  [explained_variance_score(y_train, modelo_final_ENX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_ENX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_ENX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_ENX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_ENX.predict(X_train))],
                                     'LASSO' :  [explained_variance_score(y_train, modelo_final_LASSOX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_LASSOX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_LASSOX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_LASSOX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_LASSOX.predict(X_train))],
                                     'RIDGE' :  [explained_variance_score(y_train, modelo_final_RIDGEX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_RIDGEX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_RIDGEX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_RIDGEX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_RIDGEX.predict(X_train))]                                       
                        })     
 

In [5]:
metric_results_oosZ = pd.DataFrame({'Linear Regression' :  [explained_variance_score(y_test, LRZ.predict(Z_test), multioutput='uniform_average'),
    r2_score(y_test, LRZ.predict(Z_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, LRZ.predict(Z_test)),
    mean_squared_error(y_test, LRZ.predict(Z_test)),
    mean_absolute_percentage_error(y_test, LRZ.predict(Z_test))], 
                                     'GB' :  [explained_variance_score(y_test, modelo_final_GBZ.predict(Z_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_GBZ.predict(Z_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_GBZ.predict(Z_test)),
    mean_squared_error(y_test, modelo_final_GBZ.predict(Z_test)),
    mean_absolute_percentage_error(y_test, modelo_final_GBZ.predict(Z_test))],
                                     'HGB' :  [explained_variance_score(y_test, modelo_final_HGBZ.predict(Z_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_HGBZ.predict(Z_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_HGBZ.predict(Z_test)),
    mean_squared_error(y_test, modelo_final_HGBZ.predict(Z_test)),
    mean_absolute_percentage_error(y_test, modelo_final_HGBZ.predict(Z_test))],
                                    'RF' :  [explained_variance_score(y_test, modelo_final_RFZ.predict(Z_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_RFZ.predict(Z_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_RFZ.predict(Z_test)),
    mean_squared_error(y_test, modelo_final_RFZ.predict(Z_test)),
    mean_absolute_percentage_error(y_test, modelo_final_RFZ.predict(Z_test))],
                                    'PC Regression' :  [explained_variance_score(y_test, pcrZ.predict(Z_test), multioutput='uniform_average'),
    r2_score(y_test, pcrZ.predict(Z_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, pcrZ.predict(Z_test)),
    mean_squared_error(y_test, pcrZ.predict(Z_test)),
    mean_absolute_percentage_error(y_test, pcrZ.predict(Z_test))], 
                                    'PLS' :  [explained_variance_score(y_test, plsZ.predict(Z_test), multioutput='uniform_average'),
    r2_score(y_test, plsZ.predict(Z_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, plsZ.predict(Z_test)),
    mean_squared_error(y_test, plsZ.predict(Z_test)),
    mean_absolute_percentage_error(y_test, plsZ.predict(Z_test))], 
                                    'MLP' :  [explained_variance_score(y_test, modelo_final_MLPZ.predict(Z_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_MLPZ.predict(Z_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_MLPZ.predict(Z_test)),
    mean_squared_error(y_test, modelo_final_MLPZ.predict(Z_test)),
    mean_absolute_percentage_error(y_test, modelo_final_MLPZ.predict(Z_test))],
                                    'ElasticNet' :  [explained_variance_score(y_test, modelo_final_ENZ.predict(Z_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_ENZ.predict(Z_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_ENZ.predict(Z_test)),
    mean_squared_error(y_test, modelo_final_ENZ.predict(Z_test)),
    mean_absolute_percentage_error(y_test, modelo_final_ENZ.predict(Z_test))],
                                     'LASSO' :  [explained_variance_score(y_test, modelo_final_LASSOZ.predict(Z_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_LASSOZ.predict(Z_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_LASSOZ.predict(Z_test)),
    mean_squared_error(y_test, modelo_final_LASSOZ.predict(Z_test)),
    mean_absolute_percentage_error(y_test, modelo_final_LASSOZ.predict(Z_test))],
                                     'RIDGE' :  [explained_variance_score(y_test, modelo_final_RIDGEZ.predict(Z_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_RIDGEZ.predict(Z_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_RIDGEZ.predict(Z_test)),
    mean_squared_error(y_test, modelo_final_RIDGEZ.predict(Z_test)),
    mean_absolute_percentage_error(y_test, modelo_final_RIDGEZ.predict(Z_test))]                                       
                        })    

In [6]:
metric_results_insZ = pd.DataFrame({'Linear Regression' :  [explained_variance_score(y_train, LRZ.predict(Z_train), multioutput='uniform_average'),
    r2_score(y_train, LRZ.predict(Z_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, LRZ.predict(Z_train)),
    mean_squared_error(y_train, LRZ.predict(Z_train)),
    mean_absolute_percentage_error(y_train, LRZ.predict(Z_train))], 
                                     'GB' :  [explained_variance_score(y_train, modelo_final_GBZ.predict(Z_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_GBZ.predict(Z_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_GBZ.predict(Z_train)),
    mean_squared_error(y_train, modelo_final_GBZ.predict(Z_train)),
    mean_absolute_percentage_error(y_train, modelo_final_GBZ.predict(Z_train))],
                                     'HGB' :  [explained_variance_score(y_train, modelo_final_HGBZ.predict(Z_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_HGBZ.predict(Z_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_HGBZ.predict(Z_train)),
    mean_squared_error(y_train, modelo_final_HGBZ.predict(Z_train)),
    mean_absolute_percentage_error(y_train, modelo_final_HGBZ.predict(Z_train))],
                                    'RF' :  [explained_variance_score(y_train, modelo_final_RFZ.predict(Z_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_RFZ.predict(Z_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_RFZ.predict(Z_train)),
    mean_squared_error(y_train, modelo_final_RFZ.predict(Z_train)),
    mean_absolute_percentage_error(y_train, modelo_final_RFZ.predict(Z_train))],
                                    'PC Regression' :  [explained_variance_score(y_train, pcrZ.predict(Z_train), multioutput='uniform_average'),
    r2_score(y_train, pcrZ.predict(Z_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, pcrZ.predict(Z_train)),
    mean_squared_error(y_train, pcrZ.predict(Z_train)),
    mean_absolute_percentage_error(y_train, pcrZ.predict(Z_train))], 
                                    'PLS' :  [explained_variance_score(y_train, plsZ.predict(Z_train), multioutput='uniform_average'),
    r2_score(y_train, plsZ.predict(Z_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, plsZ.predict(Z_train)),
    mean_squared_error(y_train, plsZ.predict(Z_train)),
    mean_absolute_percentage_error(y_train, plsZ.predict(Z_train))], 
                                    'MLP' :  [explained_variance_score(y_train, modelo_final_MLPZ.predict(Z_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_MLPZ.predict(Z_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_MLPZ.predict(Z_train)),
    mean_squared_error(y_train, modelo_final_MLPZ.predict(Z_train)),
    mean_absolute_percentage_error(y_train, modelo_final_MLPZ.predict(Z_train))],
                                    'ElasticNet' :  [explained_variance_score(y_train, modelo_final_ENZ.predict(Z_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_ENZ.predict(Z_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_ENZ.predict(Z_train)),
    mean_squared_error(y_train, modelo_final_ENZ.predict(Z_train)),
    mean_absolute_percentage_error(y_train, modelo_final_ENZ.predict(Z_train))],
                                     'LASSO' :  [explained_variance_score(y_train, modelo_final_LASSOZ.predict(Z_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_LASSOZ.predict(Z_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_LASSOZ.predict(Z_train)),
    mean_squared_error(y_train, modelo_final_LASSOZ.predict(Z_train)),
    mean_absolute_percentage_error(y_train, modelo_final_LASSOZ.predict(Z_train))],
                                     'RIDGE' :  [explained_variance_score(y_train, modelo_final_RIDGEZ.predict(Z_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_RIDGEZ.predict(Z_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_RIDGEZ.predict(Z_train)),
    mean_squared_error(y_train, modelo_final_RIDGEZ.predict(Z_train)),
    mean_absolute_percentage_error(y_train, modelo_final_RIDGEZ.predict(Z_train))]                                       
                        })   

In [7]:
# Change the row indexes
metric_results_oosX.index = ['Explained Variance', 'R2', 'MAE', 'MSE', 'MAPE']
metric_results_oosZ.index = ['Explained Variance', 'R2', 'MAE', 'MSE', 'MAPE']

metric_results_insX.index = ['Explained Variance', 'R2', 'MAE', 'MSE', 'MAPE']
metric_results_insZ.index = ['Explained Variance', 'R2', 'MAE', 'MSE', 'MAPE']

In [8]:

metric_results_oosX

,Linear Regression,GB,HGB,RF,PC Regression,PLS,MLP,ElasticNet,LASSO,RIDGE
Explained Variance,-2.343740e-01,5.756901e-03,2.568199e-02,-4.057831e-02,-7.726093e-02,-2.452169e-01,-2.220446e-16,-2.700171e-02,-1.927239e-01,-1.422428e-01
R2,-2.349767e-01,4.819893e-03,2.514060e-02,-4.124860e-02,-7.783584e-02,-2.455255e-01,-3.040456e-03,-2.788614e-02,-1.935548e-01,-1.429461e-01
MAE,2.858140e+04,2.593138e+04,2.375138e+04,2.463000e+04,2.556174e+04,2.807107e+04,2.479465e+04,2.557508e+04,2.789803e+04,2.719438e+04
MSE,1.792510e+10,1.444457e+10,1.414962e+10,1.511323e+10,1.564428e+10,1.807821e+10,1.455866e+10,1.491928e+10,1.732388e+10,1.658932e+10
MAPE,4.312949e+19,3.042754e+19,3.282426e+19,3.584332e+19,3.824218e+19,4.490575e+19,2.555292e+19,3.393574e+19,4.049078e+19,3.963571e+19


In [11]:

metric_results_insX

,Linear Regression,GB,HGB,RF,PC Regression,PLS,MLP,ElasticNet,LASSO,RIDGE
Explained Variance,4.818090e-01,8.475546e-02,2.266503e-01,4.696838e-01,2.044104e-01,3.892432e-01,0.000000e+00,1.188567e-01,4.591586e-01,4.490588e-01
R2,4.818090e-01,8.468013e-02,2.266496e-01,4.696837e-01,2.044104e-01,3.892432e-01,-1.045918e-03,1.188567e-01,4.591586e-01,4.490588e-01
MAE,1.953041e+04,2.173867e+04,1.804140e+04,1.934380e+04,2.027697e+04,2.071015e+04,2.121435e+04,2.117004e+04,1.960023e+04,1.965609e+04
MSE,2.417979e+09,4.271059e+09,3.608602e+09,2.474558e+09,3.712374e+09,2.849909e+09,4.671073e+09,4.111584e+09,2.523670e+09,2.570798e+09
MAPE,2.967977e+19,3.085985e+19,2.780587e+19,3.003333e+19,3.194653e+19,3.198886e+19,2.729415e+19,3.087164e+19,3.027446e+19,3.040176e+19


In [12]:

metric_results_oosZ

,Linear Regression,GB,HGB,RF,PC Regression,PLS,MLP,ElasticNet,LASSO,RIDGE
Explained Variance,2.835337e-02,9.489012e-03,6.621587e-02,-3.226580e-02,3.841113e-02,3.358932e-02,4.841440e-06,1.536531e-02,2.244450e-02,3.044054e-02
R2,2.764036e-02,8.563311e-03,6.561256e-02,-3.297225e-02,3.759324e-02,3.290093e-02,-3.657815e-02,1.412666e-02,2.126858e-02,2.972148e-02
MAE,2.659544e+04,2.559719e+04,2.395605e+04,2.386163e+04,2.580167e+04,2.652673e+04,2.323506e+04,2.520455e+04,2.483838e+04,2.645672e+04
MSE,1.411334e+10,1.439023e+10,1.356219e+10,1.499310e+10,1.396888e+10,1.403698e+10,1.504544e+10,1.430948e+10,1.420582e+10,1.408313e+10
MAPE,3.974232e+19,3.055337e+19,3.334579e+19,3.357041e+19,3.840729e+19,3.964003e+19,4.031632e+17,2.982192e+19,3.032314e+19,3.949385e+19


In [13]:

metric_results_insZ

,Linear Regression,GB,HGB,RF,PC Regression,PLS,MLP,ElasticNet,LASSO,RIDGE
Explained Variance,2.067206e-01,1.554393e-01,2.768596e-01,8.110967e-01,1.957438e-01,2.051874e-01,8.594188e-06,4.545314e-02,6.645025e-02,2.066206e-01
R2,2.067206e-01,1.553817e-01,2.768584e-01,8.103849e-01,1.957438e-01,2.051874e-01,-7.420521e-02,4.545314e-02,6.645025e-02,2.066206e-01
MAE,2.188030e+04,2.080569e+04,1.764395e+04,9.064958e+03,2.180754e+04,2.194568e+04,1.881194e+04,2.164840e+04,2.126546e+04,2.180482e+04
MSE,3.701594e+09,3.941151e+09,3.374318e+09,8.847805e+08,3.752814e+09,3.708749e+09,5.012448e+09,4.454099e+09,4.356123e+09,3.702061e+09
MAPE,3.322249e+19,2.969248e+19,2.784856e+19,1.110061e+19,3.301983e+19,3.327075e+19,4.303956e+17,3.092333e+19,3.090862e+19,3.309251e+19


In [15]:
metric_results_oosX.to_clipboard()

In [ ]:
print('Finish')

In [17]:
df_pred = pd.read_csv(path + '/_aux/p3_case_value.csv') 
predX = df_pred[['gen',
            'reclutamiento',
            'salario_diario',
            'horas_sem',
            'reinst',
            'prima_dom',
            'desc_sem',
            'sarimssinf',
            'antiguedad',
            'c_recsueldo',
            'desc_ob',
            'min_prima_antig',
            'min_ag',
            'min_vac',
            'min_ley']]

predZ = df_pred[['num_actores',
            'gen',
            'reclutamiento',
            'salario_diario',
            'antiguedad',          
            'horas_sem',
            'reinst',
            'indem',
            'sal_caidos',
            'prima_antig',
            'prima_vac',
            'horas_extra',
            'rec20',
            'prima_dom',
            'desc_sem',
            'sarimssinf',
            'utilidades',
            'nulidad']]


dict_modelsX = {
    "GB": [modelo_final_GBX, GBX],
    "HGB": [modelo_final_HGBX, HGBX],
    "RF": [modelo_final_RFX, RFX]
}

dict_modelsZ = {
    "GB": [modelo_final_GBZ, GBZ],
    "HGB": [modelo_final_HGBZ, HGBZ],
    "RF": [modelo_final_RFZ, RFZ]
}


In [28]:
case_value = df_pred[['id_actor', 'dem']]





In [29]:
for model in dict_modelsX:
    params = dict_modelsX[model][0].get_params()
    dict_modelsX[model][1].set_params(**params)
    dict_modelsX[model][1].fit(X,y2)
    pre = pd.DataFrame(dict_modelsX[model][1].predict(predX))
    name = 'preX_'+model
    pre.rename(columns={0: name}, inplace=True)
    case_value = pd.concat([case_value, pre], axis=1)

In [30]:
for model in dict_modelsZ:
    params = dict_modelsZ[model][0].get_params()
    dict_modelsZ[model][1].set_params(**params)
    dict_modelsZ[model][1].fit(Z,y2)
    pre = pd.DataFrame(dict_modelsZ[model][1].predict(predZ))
    name = 'preZ_'+model
    pre.rename(columns={0: name}, inplace=True)
    case_value = pd.concat([case_value, pre], axis=1)

In [31]:
case_value

,id_actor,dem,preX_GB,preX_HGB,preX_RF,preZ_GB,preZ_HGB,preZ_RF
0,100,0,27711.263221,23891.365194,15159.673135,18350.893952,20280.280089,12947.882439
1,1000,0,14241.634424,15881.867273,11465.052658,15328.599842,14168.255732,1751.510616
2,1000_1,0,18637.475058,17291.948941,12788.842459,17902.191585,16519.780982,8035.597409
3,1001,0,22970.684514,18972.508864,18814.369245,23188.004696,20680.558165,19652.144715
4,1001_1,0,16209.037857,17291.948941,11856.464822,15415.054025,15657.642393,5522.309140
...,...,...,...,...,...,...,...,...
4149,889,1,22884.981655,18293.693958,24019.690656,22751.858437,17940.484477,22629.845395
4150,91,1,20457.276448,20120.488736,13627.953228,18158.982323,21248.016080,10987.297812
4151,959_1,1,17508.592861,19446.885543,14120.860890,19602.185285,17776.988087,14461.967766
4152,968,1,17359.974086,16048.699123,11465.052658,18229.763040,14995.221808,7576.413293


In [32]:
import os
case_value.to_csv(os.path.join(path + '/_aux/case_value.csv') , index = False)